## Multi Container endpoints

In this example we will deploy two different models for summarization and Q&A tasks.
Please note that loading and packaging models may take several minutes.

In [100]:
# Download artifacts for DistilBert model for Question-Answering task

! mkdir distilbert-base-uncased-distilled-squad
! mkdir distilbert-base-uncased-distilled-squad/1
! mkdir distilbert-base-uncased-distilled-squad/code

! wget https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/saved_model.tar.gz
! tar -zxvf saved_model.tar.gz -C distilbert-base-uncased-distilled-squad/1

! cp 1_src/inference.py distilbert-base-uncased-distilled-squad/code
! cp 1_src/requirements.txt distilbert-base-uncased-distilled-squad/code

# Old model fetch. to be deleted.
#! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/tf_model.h5 -P distilbert-base-uncased-distilled-squad
#! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/tokenizer.json -P distilbert-base-uncased-distilled-squad
#! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/tokenizer_config.json -P distilbert-base-uncased-distilled-squad
#! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/vocab.txt -P distilbert-base-uncased-distilled-squad
#! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/config.json -P distilbert-base-uncased-distilled-squad



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [101]:
!tar -C "$PWD" -czf distilbert-base-uncased-distilled-squad.tar.gz distilbert-base-uncased-distilled-squad/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [103]:
# Download artifacts for Bart model for Summarization task

! wget https://huggingface.co/facebook/bart-large-cnn/resolve/main/pytorch_model.bin -P bart-large-cnn
! wget https://huggingface.co/facebook/bart-large-cnn/resolve/main/tokenizer.json -P bart-large-cnn
! wget https://huggingface.co/facebook/bart-large-cnn/resolve/main/vocab.json -P bart-large-cnn
! wget https://huggingface.co/facebook/bart-large-cnn/resolve/main/config.json -P bart-large-cnn
! wget https://huggingface.co/facebook/bart-large-cnn/resolve/main/merges.txt -P bart-large-cnn
! cp 1_src/inference.py bart-large-cnn
! cp 1_src/requirements.txt bart-large-cnn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--2022-08-13 18:48:20--  https://huggingface.co/facebook/bart-large-cnn/resolve/main/pytorch_model.bin
Resolving huggingface.co (huggingface.co)... 52.6.16.131, 34.231.117.252
Connecting to huggingface.co (huggingface.co)|52.6.16.131|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/facebook/bart-large-cnn/2ac2745c02ac987d82c78a14b426de58d5e4178ae8039ba1c6881eccff3e82f1?response-content-disposition=attachment%3B%20filename%3D%22pytorch_model.bin%22 [following]
--2022-08-13 18:48:20--  https://cdn-lfs.huggingface.co/facebook/bart-large-cnn/2ac2745c02ac987d82c78a14b426de58d5e4178ae8039ba1c6881eccff3e82f1?respon

In [104]:
!tar -C "$PWD" -czf bart-large-cnn.tar.gz bart-large-cnn/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Upload model data to S3

In [ ]:

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
#role = get_execution_role()  # TODO: replace it
role="arn:aws:iam::941656036254:role/service-role/AmazonSageMaker-ExecutionRole-20210904T193230" # TODO: this has to be replaced

bucket = sagemaker_session.default_bucket()
prefix = 'multi-container'
s3_path = 's3://{}/{}'.format(bucket, prefix)


In [106]:
qa_model_data = sagemaker_session.upload_data('distilbert-base-uncased-distilled-squad.tar.gz',
                                           bucket,
                                           os.path.join(prefix, 'model-artifacts'))

summarization_model_data = sagemaker_session.upload_data('bart-large-cnn.tar.gz',
                                           bucket,
                                           os.path.join(prefix, 'model-artifacts'))                                           

# Create Inference script

In [ ]:
! pygmentize 1_src/inference.py

In [121]:
### For TensorFlow we need to check signature of the model

! saved_model_cli show --dir distilbert-base-uncased-distilled-squad/1 --tag_set serve --signature_def serving_default

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The given SavedModel SignatureDef contains the following input(s):
  inputs['attention_mask'] tensor_info:
      dtype: DT_INT32
      shape: (-1, 384)
      name: serving_default_attention_mask:0
  inputs['input_ids'] tensor_info:
      dtype: DT_INT32
      shape: (-1, 384)
      name: serving_default_input_ids:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['output_0'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 384)
      name: StatefulPartitionedCall:0
  outputs['output_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 384)
      name: StatefulPartitionedCall:1
Method name is: tensorflow/serving/predict


## Local mode

TODO: delete

In [ ]:
from sagemaker.pytorch import PyTorchModel

env = { "NLP_TASK":"summarization"
    }


# Note: You can update the 'torchserve-predictor.py' file as needed according to the model you want to use (ie BERT) 
model = PyTorchModel(model_data=summarization_model_data,
                   role=role, 
                   entry_point='pipeline_predictor.py',
                   source_dir='1_src',
                   framework_version='1.9.0',
                   py_version='py38',
                   env=env,
                   sagemaker_session=sagemaker_local_session)

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

local_predictor = model.deploy(initial_instance_count=1, instance_type=instance_type, serializer=JSONSerializer(), deserializer=JSONDeserializer())

In [ ]:
import json

context = r"""
The Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain "Amazonas" in their names. The Amazon represents over half of the planet's remaining rainforests, and comprises the largest and most biodiverse tract of tropical rainforest in the world, with an estimated 390 billion individual trees divided into 16,000 species.
"""

question="What kind of forest is Amazon?"


data = {"article":context, "max_length":100}
print(data)
#local_predictor.predict(data)


# Deploy Multi Container Endpoint


In [108]:
region = sagemaker_session.boto_region_name

instance_type = "ml.c5.4xlarge"

In [109]:
summarization_env = {
    "NLP_TASK" : "summarization",
    "SAGEMAKER_PROGRAM" : "inference.py",
    "SAGEMAKER_SUBMIT_DIRECTORY": summarization_model_data,
}

pt_inference_image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=region,
    version="1.9",
    py_version="py38",
    instance_type=instance_type,
    image_scope="inference",
)

pytorch_container = {
    "ContainerHostname": "pytorch-bart-summarizer",
    "Image": pt_inference_image_uri,
    "ModelDataUrl": summarization_model_data,
    "Environment" : summarization_env
}


In [110]:
qa_env = {
    "NLP_TASK" : "question-answering"

}

tf_inference_image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.8",
    py_version="py38",
    instance_type=instance_type,
    image_scope="inference",
)

tensorflow_container = {
    "ContainerHostname": "tensorflow-distilbert-qa",
    "Image": tf_inference_image_uri,
    "ModelDataUrl": qa_model_data,
    "Environment" : qa_env
}


In [111]:
print(pytorch_container)
print(tensorflow_container)

{'ContainerHostname': 'pytorch-bart-summarizer', 'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.9-cpu-py38', 'ModelDataUrl': 's3://sagemaker-us-east-1-941656036254/multi-container/model-artifacts/bart-large-cnn.tar.gz', 'Environment': {'NLP_TASK': 'summarization', 'SAGEMAKER_PROGRAM': 'inference.py', 'SAGEMAKER_SUBMIT_DIRECTORY': 's3://sagemaker-us-east-1-941656036254/multi-container/model-artifacts/bart-large-cnn.tar.gz'}}
{'ContainerHostname': 'tensorflow-distilbert-qa', 'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.8-cpu', 'ModelDataUrl': 's3://sagemaker-us-east-1-941656036254/multi-container/model-artifacts/distilbert-base-uncased-distilled-squad.tar.gz', 'Environment': {'NLP_TASK': 'question-answering'}}


## Create Multi Container Endpoint

In [112]:
sm_client = sagemaker_session.sagemaker_client
runtime_sm_client = sagemaker_session.sagemaker_runtime_client



In [113]:
import datetime

unique_id = datetime.datetime.now().strftime("%Y-%m-%d%H-%M-%S")

model_name = f"nlp-multi-container-{unique_id}"

create_model_response = sm_client.create_model(
    ModelName=model_name,
    Containers=[pytorch_container, tensorflow_container],
    InferenceExecutionConfig={"Mode": "Direct"},
    ExecutionRoleArn=role,
)

In [114]:
endpoint_config_name = f"nlp-multi-container-ep-config-{unique_id}"

endpoint_config = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "prod",
            "ModelName": "nlp-multi-container",
            "InitialInstanceCount": 1,
            "InstanceType": instance_type,
        },
    ],
)

In [115]:
endpoint_name = f"nlp-multi-container-ep-{unique_id}"

endpoint = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

## Testing Multi Container Endpoint

In [122]:
import json

article = r"""
The Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain "Amazonas" in their names. The Amazon represents over half of the planet's remaining rainforests, and comprises the largest and most biodiverse tract of tropical rainforest in the world, with an estimated 390 billion individual trees divided into 16,000 species.
"""

question="What kind of forest is Amazon?"

#qa_input = {"question":question, "context":article}


In [123]:
summarization_input = {"article":article, "max_length":100}

In [168]:
#  preparing data for TF Serving format

from transformers import DistilBertTokenizer, TFDistilBertForQuestionAnswering
import tensorflow as tf
import numpy as np

max_length = 384
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")
model = TFDistilBertForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")

encoded_input = tokenizer(question, article, padding='max_length', max_length=max_length)
encoded_input = dict(encoded_input)
qa_inputs = [{"input_ids": np.array(encoded_input["input_ids"]).tolist(), "attention_mask":np.array(encoded_input["attention_mask"]).tolist()}]
qa_inputs = {"instances" : qa_inputs}
#outputs = model(**inputs)

#answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
#answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])

#predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
#tokenizer.decode(predict_answer_tokens)

Some layers from the model checkpoint at distilbert-base-cased-distilled-squad were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased-distilled-squad and are newly initialized: ['dropout_179']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [169]:
print(qa_inputs)

{'instances': [{'input_ids': [101, 1327, 1912, 1104, 3304, 1110, 9786, 136, 102, 1109, 9786, 24369, 113, 4269, 131, 18827, 1777, 7277, 10961, 28206, 13109, 1137, 7277, 10961, 28206, 5813, 132, 2124, 131, 22087, 1233, 2497, 7277, 10961, 4722, 4578, 117, 9786, 7171, 1137, 1932, 9786, 1465, 132, 1497, 131, 1370, 24559, 1204, 1821, 10961, 11153, 26042, 132, 2954, 131, 9786, 9014, 4915, 12821, 4867, 114, 117, 1145, 1227, 1107, 1483, 1112, 9786, 1465, 1137, 1103, 9786, 17009, 117, 1110, 170, 11758, 4728, 21407, 3304, 1115, 3662, 1211, 1104, 1103, 9786, 8434, 1104, 1375, 1738, 119, 1188, 8434, 14474, 128, 117, 1288, 117, 1288, 1961, 2850, 113, 123, 117, 5689, 117, 1288, 4816, 1940, 114, 117, 1104, 1134, 126, 117, 2260, 117, 1288, 1961, 2850, 113, 123, 117, 1620, 117, 1288, 4816, 1940, 114, 1132, 2262, 1118, 1103, 24369, 119, 1188, 1805, 2075, 3441, 7078, 1106, 2551, 6015, 119, 1109, 2656, 1104, 1103, 3304, 1110, 4049, 1439, 3524, 117, 1114, 2539, 110, 1104, 1103, 24369, 117, 1723, 1118, 7022,

In [170]:
import numpy as np

tf_response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    TargetContainerHostname="tensorflow-distilbert-qa",
    Body=json.dumps(qa_inputs),
)

In [171]:
predictions = json.loads(tf_response["Body"].read().decode())

In [179]:
print(predictions['predictions'][0]['output_0'])
print(tf.math.argmax(predictions['predictions'][0]['output_0']))

[-2.81213641, -6.33242798, -7.15034676, -7.79426241, -6.90369606, -8.3878746, -5.81328297, -6.05798721, -6.61176205, 1.79013884, 4.34852314, -0.252203882, -4.39401484, -4.04632378, -7.97626305, -3.93852258, -7.52109814, -5.39915323, -7.81546879, -8.23505402, -7.76532936, -7.82198906, -4.54771233, -8.06315, -8.57569695, -7.88697243, -7.86777449, -4.8922739, -7.9327507, -4.11743212, -7.51967812, -7.50865, -6.01840925, -7.89440441, -7.91009808, -7.60474825, -7.92939043, -3.12175298, -7.23803806, -8.49301, -5.9343586, -2.62836814, -7.25240135, -7.82351112, -5.01983881, -7.87251902, -5.06236935, -7.2889185, -7.07985878, -6.05922174, -7.75251818, -7.95574093, -7.72836256, -7.57692575, -4.79824066, -7.64635849, -2.57327676, -7.0719595, -7.66239166, -7.45005035, -7.40450716, -4.3102684, -5.45500469, -5.99215889, -6.43772, -5.71161842, -5.73724747, -7.01345539, 0.741526544, -6.04399204, -7.55634308, -0.873561263, 2.05227709, -3.75576329, -4.57667065, -0.241587117, 5.76309681, 9.03434181, 8.1337

In [187]:
answer_start_index = int(tf.math.argmax(predictions['predictions'][0]['output_0']))
answer_end_index = int(tf.math.argmax(predictions['predictions'][0]['output_1']))

predict_answer_tokens = encoded_input["input_ids"][answer_start_index : answer_end_index + 1]
tf_response = tokenizer.decode(predict_answer_tokens)

print(f"Question: {question}, answer: {tf_response}")


Question: What kind of forest is Amazon?, answer: moist broadleaf forest


In [ ]:
pt_result = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    TargetContainerHostname="pytorch-bart-summarizer",
    Body=json.dumps(summarization_input),
)

# Below are initial experiments.

To be removed

In [ ]:
from sagemaker.pytorch import PyTorchModel

env = {
    "NLP_TASK": "summarization"
    }


# Note: You can update the 'torchserve-predictor.py' file as needed according to the model you want to use (ie BERT) 
model = PyTorchModel(model_data=summarization_model_data,
                   role=role, 
                   entry_point='pipeline_predictor.py',
                   source_dir='1_src',
                   framework_version='1.9.0',
                   py_version='py38',
                   env=env,
                   sagemaker_session=sagemaker_session)

In [ ]:
remote_predictor = model.deploy(initial_instance_count=1, instance_type="ml.g4dn.4xlarge", serializer=JSONSerializer(), deserializer=JSONDeserializer())

In [ ]:
remote_predictor.predict(data)

In [ ]:
from time import perf_counter

def timer(f,*args):   
    
    start = perf_counter()
    f(*args)
    return (1000 * (perf_counter() - start))


stats = np.array([timer(predict_f, local_predictor) for _ in range(10)])
print(stats)

for i in range(100):
    local_predictor.predict(data)

In [ ]:
local_predictor.endpoint_name